## 1. Initialization

In [1]:
import sys
import os

os.chdir("../")
path1 = os.getcwd()

os.chdir("coppeliasim_utils")
path2 = os.getcwd()


sys.path.append(path1)
sys.path.append(path2)

from indy_utils import indydcp_client as client
from coppeliasim_utils import sim

from time import sleep
import time
import json
import threading
import numpy as np
import math
import keyboard
import threading

robot_ip = "192.168.1.6"  # Robot (Indy) IP
robot_name = "NRMK-Indy7"  # Robot name (Indy7)

# Create class object (real robot)
indy_actual = client.IndyDCPClient(robot_ip, robot_name)

# Crate class object (simulation robot)
indy_virtual = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)

# Actual robot (Created: True, Not created: False)
# Virtual robot (Created: True, Not created: False)
print("real robot: {0}\nsimulation robot: {1}".format(bool(indy_actual), bool(not indy_virtual)))

ModuleNotFoundError: No module named 'keyboard'

## 2. Communication reset

In [11]:
# Reset TCP-IP communication (real robot)
# For real robot
indy_actual.connect()
indy_actual.disconnect()

# For simulation robot
sim.simxFinish(indy_virtual)

Connect: Server IP (192.168.1.6)


## 3. Position data acquisition

In [8]:
# For real robot
indy_actual.connect()
j_pos_act = indy_actual.get_joint_pos()
t_pos_act = indy_actual.get_task_pos()

# For simulation robot
handles = []
j_pos_vir = []
t_pos_vir = []

for i in range(6):
    object_name = 'joint' + str(i)
    result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
    if result != sim.simx_return_ok:
        raise Exception('could not get object handle for first joint')                
    else:
        handles.append(handle)
        result, jnt_angle = sim.simxGetJointPosition(indy_virtual, handle, sim.simx_opmode_oneshot)
        j_pos_vir.append(jnt_angle/math.pi*180)
        
object_name = 'tool_coordinate'
result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)

result, obj_pos = sim.simxGetObjectPosition(indy_virtual, handle, -1, sim.simx_opmode_oneshot)
result, obj_rot = sim.simxGetObjectOrientation(indy_virtual, handle, -1, sim.simx_opmode_oneshot)

print()

for i in range(3):
    t_pos_vir.append(obj_pos[i])

for i in range(3):
    t_pos_vir.append(obj_rot[i]/np.pi*180)
        
print("Joint Position (real):", np.round(j_pos_act, 2))
print("Joint Position (simulation):", np.round(j_pos_vir, 2))

print()
print("Task Position (real):", np.round(t_pos_act,2))
print("Task Position (simulation):", np.round(t_pos_vir,2))


Connect: Server IP (192.168.1.6)

Joint Position (real): [ -0. -15. -90.  -0. -75.   0.]
Joint Position (simulation): [  0. -15. -90.   0. -75.   0.]

Task Position (real): [  0.45  -0.19   0.42   0.   180.     0.  ]
Task Position (simulation): [   0.45   -0.19    0.42 -180.     -0.   -180.  ]


## 4. Set position

In [124]:
### For actual robot
indy_actual.connect()

# Zero position
#j_pos_act = [0, 0, 0, 0, 0, 0]

# Home position
j_pos_act = [0, -15, -90, 0, -75, 0]
indy_actual.joint_move_to(j_pos_act)  # Move 6th joint

indy_actual.disconnect()

### For virtual robot
handles = []
#Zero position
#j_pos_vir = [0, 0, 0, 0, 0, 0]

# Home position
j_pos_vir = [0, -15, -90, 0, -75, 0]

for i in range(6):
    object_name = 'joint' + str(i)
    #print(object_name)
    result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
    if result != sim.simx_return_ok:
        raise Exception('could not get object handle for first joint')                
    else:
        handles.append(handle)
        sim.simxSetJointTargetPosition(indy_virtual, handle, j_pos_vir[i]*math.pi/180, sim.simx_opmode_oneshot)

Connect: Server IP (192.168.1.6)


## 5. Synchronization

### 5-1. Synchronization (w non threaded)

In [2]:
indy_actual.connect()

handles = []

for i in range(6):
        object_name = 'joint' + str(i)    
        result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
        handles.append(handle)
        
print(handles)

isCtrOn = True

while isCtrOn:
    j_pos = indy_actual.get_joint_pos()
    time.sleep(0.002)

    for i in range(6):
        sim.simxSetJointTargetPosition(indy_virtual, handles[i] , float(j_pos[i])*math.pi/180 ,sim.simx_opmode_oneshot)
    
    if keyboard.is_pressed("q"):
        isCtrOn = False
        print("Escape the program")

indy_actual.disconnect()

Connect: Server IP (192.168.1.6)
[19, 21, 23, 25, 27, 29]
Escape the program


### 5-2. Synchronization (w threaded)

In [3]:
def cps_program():
    indy_actual.connect()

    handles = []

    for i in range(6):
            object_name = 'joint' + str(i)    
            #print(object_name)
            result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
            handles.append(handle)

    print(handles)

    isCtrOn = True

    while isCtrOn:
        j_pos = indy_actual.get_joint_pos()
        time.sleep(0.002)

        for i in range(6):
            sim.simxSetJointTargetPosition(indy_virtual, handles[i] , float(j_pos[i])*math.pi/180 ,sim.simx_opmode_oneshot)

        if keyboard.is_pressed("q"):
            isCtrOn = False
            print("Escape the program")

    indy_actual.disconnect()

t = threading.Thread(target=cps_program)
t.start()
print("Main Thread")

Main Thread
Connect: Server IP (192.168.1.6)
[19, 21, 23, 25, 27, 29]


### 5-3. Motion test

In [6]:
j_pos1 = indy_actual.get_joint_pos()
t_pos1 = indy_actual.get_task_pos()
print("j_pos1", j_pos1)
print("t_pos1", t_pos1)

t_pos1[2] += 0.1
indy_actual.task_move_to(t_pos1)  # Move along z-axis

j_pos1 [-2.788240125940038e-15, -13.441191131129374, -75.51009058405377, -9.526175978467308e-15, -91.04871828482023, 2.02100007092053e-15]
t_pos1 [0.45454260949727554, -0.18650000000000008, 0.5160799560441868, 2.3683643230183005e-14, -179.99999999999665, -5.106281148755881e-15]
Escape the program


In [11]:
indy_actual.go_home()

In [7]:
indy_actual.stop_motion()

In [10]:
indy_actual.go_zero()